<br>

# Introdução

Extrai os arquivos baixados em .rar e transforma em .gpkg

In [ ]:
#!pip3 install rarfile --upgrade

In [ ]:
import os
import time
import shutil
import geopandas as gpd
from rarfile import RarFile

In [ ]:
from paths import *

In [ ]:
list_files = os.listdir(bruto_path)
list_files = [x for x in list_files if x.endswith('.rar')]

# 
print(f'Temos {len(list_files)} arquivos .rar')
#list_files = list_files[-1:]
list_files

In [ ]:
def write_geodata(output_path, filename):
    gdf.to_file(
        os.path.join(output_path, 'comite_alto_tiete.gpkg'),
        layer=f'{filename}',
        driver='GPKG'
    )

    gdf.to_file(
        os.path.join(output_path, f'{filename}.gpkg'),
        layer=f'{filename}',
        driver='GPKG'
    )

In [ ]:
def delete_path(path, msg):
    os.chmod(path, 0o777)
    shutil.rmtree(
        path,
        onerror=print(msg),
        ignore_errors=True,
    )

In [ ]:
def unrar_files(rar_filepath, output_path):
    # Unrar data
    with RarFile(rar_filepath) as rarObj:
        #for f in rarObj.infolist():
            #print(f'{f.filename}\n{f.file_size}\n{f.is_dir()}\n{f.is_file()}\n')
        rarObj.extractall(output_path)

    # Se extrar pasta, move conteudo da pasta
    for root, dirs, files in os.walk(output_path, topdown=False):
        for file in files:
            try:
                shutil.move(os.path.join(root, file), output_path)
            except OSError:
                pass

In [ ]:
for file in list_files:
    print('-'*40)
    print(f'{file}')
    rarfile_path = os.path.join(bruto_path, file)

    # Set path to work
    temp_path = os.path.join(os.path.dirname(rarfile_path), 'temp')
    os.makedirs(temp_path, exist_ok=True)
    
    # Delete Path
    delete_path(temp_path, '')
    
    # Unrar data
    unrar_files(rarfile_path, temp_path)
  
    # Get list of shapefiles
    list_shps = [x for x in os.listdir(temp_path) if x.endswith('.shp')]
    #print(list_shps)

    if len(list_shps) == 1:
        # Shapefile
        shp_file = list_shps[0]
        #print(shp_file)
        
        # Set name for output
        shp_file_out = shp_file.replace('.shp', '').lower()
        print(shp_file_out)
        
        # Read Shapefile
        if shp_file_out == '45_subbacias': 
            gdf = gpd.read_file(
                os.path.join(temp_path, shp_file),
                encoding='ISO-8859-1',
            ).to_crs(epsg=4326)
            dict_fix = {
                'CrÃ\xadtica': 'Crítica',
                'NÃ£o CrÃ\xadtica': 'Não Crítica',
                'Sem ClassificaÃ§Ã£o': 'Sem Classificação'
            }
            gdf.replace(
                {
                    'Crit_IQA': dict_fix,
                    'Crit_Uso': dict_fix,
                    'IAP': dict_fix,
                    'IVA': dict_fix,
                    'OD': dict_fix,
                },
                inplace=True
            )            
            
        else:
            gdf = gpd.read_file(
                os.path.join(temp_path, shp_file),
            ).to_crs(epsg=4326)
            
        # Write data
        write_geodata(output_path_geo, shp_file_out)

        # Delete Path
        delete_path(temp_path, 'Erro pra Deletar!')
        time.sleep(0.5)

    else:
        print('deu ruim!!!')
    print('   ')